In [1]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [16]:
DROP Table IF EXISTS idade;
DROP Table IF EXISTS Media;
DROP Table IF EXISTS Contagem;
DROP Table IF EXISTS Contagem2;

DROP Table IF EXISTS MaxCount2;
DROP Table IF EXISTS MaxCount;
DROP Table IF EXISTS DrugCount2;
DROP Table IF EXISTS DrugCount;
DROP Table IF EXISTS Person;
DROP Table IF EXISTS Drug;
DROP Table IF EXISTS DrugUse;

In [17]:
CREATE TABLE Person (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../data/demographic-person.csv');

CREATE TABLE Drug (
  Code VARCHAR(6) NOT NULL,
  Name VARCHAR(100) NOT NULL,
  PRIMARY KEY(Code)
) AS SELECT
    code,
    name
FROM CSVREAD('../data/medications-drug.csv');

CREATE TABLE DrugUse (
  PersonId VARCHAR(5) NOT NULL,
  DrugCode VARCHAR(6) NOT NULL,
  DaysUse INTEGER,
  FOREIGN KEY(PersonId)
    REFERENCES Person(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(DrugCode)
    REFERENCES Drug(Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../data/medications-use.csv');

In [29]:
SELECT * FROM Person;
SELECT * FROM Drug;
SELECT * FROM DrugUse

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [23]:
SELECT Drug.Name , COUNT(DrugUse.PersonID) FROM Drug , DrugUse
    WHERE Drug.Code=DrugUse.DrugCode
    GROUP BY Drug.Name

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [58]:
CREATE VIEW DrugCount AS
    SELECT DrugCode , COUNT(PersonId) Number FROM DrugUse
        GROUP BY DrugCode;
CREATE VIEW MaxCount AS
    SELECT MAX(Number) Maximum FROM DrugCount;

SELECT DrugCount.DrugCode FROM DrugCount,MaxCount
    WHERE DrugCount.Number=MaxCount.Maximum;

d00732

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [60]:
SELECT DrugCode FROM
    (SELECT DrugCode , COUNT(PersonId) Number FROM DrugUse
        GROUP BY DrugCode) DrugCount,
    (SELECT MAX(Number) Maximum FROM (SELECT DrugCode , COUNT(PersonId) Number FROM DrugUse
        GROUP BY DrugCode)) MaxCount
    
    WHERE MaxCount.Maximum=DrugCount.Number;

d00732

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [66]:
CREATE VIEW DrugCount2 AS
    SELECT Drug.Name Name, COUNT(DrugUse.PersonId) Number FROM DrugUse,Drug
        WHERE DrugUse.DrugCode=Drug.Code
        GROUP BY DrugCode;
CREATE VIEW MaxCount2 AS
    SELECT MAX(Number) Maximum FROM DrugCount2;

SELECT DrugCount2.Name FROM DrugCount2,MaxCount2
    WHERE DrugCount2.Number=MaxCount2.Maximum;

LISINOPRIL

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [8]:
SELECT AVG(Numero) FROM
    (SELECT COUNT(PersonID) Numero FROM DrugUse
         GROUP BY PersonID);

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [9]:
CREATE VIEW Contagem AS SELECT PersonID,COUNT(PersonID) Numero FROM DrugUse
         GROUP BY PersonID;
CREATE VIEW Media AS SELECT AVG(Numero) Media FROM Contagem;
SELECT Contagem.PersonID FROM Contagem,Media
    WHERE Contagem.Numero>Media.Media;

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [18]:
CREATE VIEW Contagem2 AS SELECT PersonID,COUNT(PersonID) Numero FROM DrugUse
         GROUP BY PersonID;

CREATE VIEW Idade AS SELECT Person.ID ID, Person.Age/10*10 Faixa,Contagem2.Numero Numero FROM Person,Contagem2
    WHERE Contagem2.PersonID=Person.ID;

SELECT Faixa, AVG(Numero) FROM Idade
     GROUP BY Faixa;

# Navegando por Hierarquias

A seguir exercícios envolvendo navegação por hierarquias.

# Marcadores e Taxonomia

Considere o modelo para `Marcadores` e `Taxonomia`:

![UML](labs/2021/04-sql-advanced/marcadores-taxonomia-uml.png)

![Relacional](labs/2021/04-sql-advanced/marcadores-taxonomia-er.png)

Considere as sentenças SQL a seguir para montar as tabelas de `Marcadores` e `Taxonomia`:

In [2]:
DROP TABLE IF EXISTS Taxonomia;
DROP TABLE IF EXISTS Marcadores;

CREATE TABLE Taxonomia (
  Categoria VARCHAR(50) NOT NULL,
  Superior  VARCHAR(50),
  PRIMARY KEY(Categoria),
  FOREIGN KEY(Superior)
    REFERENCES Taxonomia(Categoria)
      ON DELETE NO ACTION
      ON UPDATE CASCADE
);

CREATE TABLE Marcadores (
  Titulo VARCHAR(50) NOT NULL,
  Endereco VARCHAR(80) NOT NULL,
  Acessos INTEGER,
  Categoria VARCHAR(50) NOT NULL,
  PRIMARY KEY(Titulo),
  FOREIGN KEY(Categoria)
    REFERENCES Taxonomia(Categoria)
      ON DELETE NO ACTION
      ON UPDATE CASCADE
);

As tabelas são preenchidas com as seguintes tuplas:

In [3]:
INSERT INTO Taxonomia VALUES ('Geral', NULL);
INSERT INTO Taxonomia VALUES ('Serviços', 'Geral');
INSERT INTO Taxonomia VALUES ('Acadêmico', 'Geral');
INSERT INTO Taxonomia VALUES ('Relacionamento', 'Geral');
INSERT INTO Taxonomia VALUES ('Busca', 'Serviços');
INSERT INTO Taxonomia VALUES ('Portal', 'Serviços');
INSERT INTO Taxonomia VALUES ('Mail', 'Serviços');
INSERT INTO Taxonomia VALUES ('Vendas', 'Serviços');
INSERT INTO Taxonomia VALUES ('Universidade', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('CG', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('Sociedade', 'Acadêmico');
INSERT INTO Taxonomia VALUES ('Rede Social', 'Relacionamento');
INSERT INTO Taxonomia VALUES ('Marketplace', 'Vendas');
INSERT INTO Taxonomia VALUES ('Loja', 'Vendas');
INSERT INTO Taxonomia VALUES ('Notícias', 'Portal');
INSERT INTO Taxonomia VALUES ('Agregador', 'Portal');

INSERT INTO Marcadores VALUES ('Terra', 'http://www.terra.com.br', 295, 'Portal');
INSERT INTO Marcadores VALUES ('POVRay', 'http://www.povray.org', 2, 'CG');
INSERT INTO Marcadores VALUES ('SBC', 'http://www.sbc.org.br', 26, 'Sociedade');
INSERT INTO Marcadores VALUES ('Correios', 'http://www.correios.com.br', 45, 'Serviços');
INSERT INTO Marcadores VALUES ('GMail', 'http://www.gmail.com', 296, 'Mail');
INSERT INTO Marcadores VALUES ('Google', 'http://www.google.com', 1590, 'Busca');
INSERT INTO Marcadores VALUES ('Yahoo', 'http://www.yahoo.com', 134, 'Serviços');
INSERT INTO Marcadores VALUES ('Orkut', 'http://www.orkut.com', 45, 'Rede Social');
INSERT INTO Marcadores VALUES ('iBahia', 'http://www.ibahia.com', 3, 'Portal');
INSERT INTO Marcadores VALUES ('Submarino', 'http://www.submarino.com.br', 320, 'Loja');
INSERT INTO Marcadores VALUES ('Amazon', 'https://www.amazon.com.br', 410, 'Marketplace');
INSERT INTO Marcadores VALUES ('Americanas', 'https://www.americanas.com.br', 320, 'Loja');
INSERT INTO Marcadores VALUES ('Mercado Livre', 'https://www.mercadolivre.com.br', 500, 'Marketplace');
INSERT INTO Marcadores VALUES ('G1', 'https://g1.globo.com', 1200, 'Notícias');
INSERT INTO Marcadores VALUES ('Folha', 'https://www.folha.uol.com.br', 850, 'Notícias');
INSERT INTO Marcadores VALUES ('Google News', 'https://news.google.com', 900, 'Agregador');
INSERT INTO Marcadores VALUES ('Flipboard', 'https://flipboard.com', 950, 'Agregador');

Conteúdo da tabela `Taxonomia`:

In [23]:
SELECT * FROM Taxonomia;

Conteúdo da tabela `Marcadores`:

In [24]:
SELECT * FROM Marcadores;

## Tarefa 1

Escreva em SQL uma consulta que retorne os marcadores da categoria `Serviços`, sem considerar as categorias subordinadas.

In [6]:
SELECT Titulo FROM Marcadores
    WHERE Categoria='Serviços';

## Tarefa 2

Escreva em SQL uma consulta que retorne os marcadores da categoria `Serviços`, considerando as categorias subordinadas.

In [ ]:
DROP TABLE IF EXISTS Servicos3;
DROP TABLE IF EXISTS Servicos2;
DROP TABLE IF EXISTS Servicos1;

DROP TABLE IF EXISTS Servicos0;


CREATE VIEW Servicos0 AS SELECT Categoria FROM Taxonomia
    WHERE Categoria='Serviços';

CREATE VIEW Servicos1 AS (SELECT Taxonomia.Categoria Categoria FROM Servicos0,Taxonomia
    WHERE Taxonomia.Superior=Servicos0.Categoria)
    UNION (SELECT * FROM Servicos0);
CREATE VIEW Servicos2 AS (SELECT Taxonomia.Categoria Categoria FROM Servicos1,Taxonomia
    WHERE Taxonomia.Superior=Servicos1.Categoria)
    UNION (SELECT * FROM Servicos1);
CREATE VIEW Servicos3 AS (SELECT Taxonomia.Categoria Categoria FROM Servicos2,Taxonomia
    WHERE Taxonomia.Superior=Servicos2.Categoria)
    UNION (SELECT * FROM Servicos2);



SELECT Marcadores.Titulo FROM Servicos3,Marcadores
    WHERE Marcadores.Categoria=Servicos3.Categoria;
    